In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration

/opt/anaconda3/envs/stable_env_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/stable_env_nlp/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/anaconda3/envs/stable_env_nlp/lib/python3.10/

In [2]:
df = pd.read_csv('/Users/abhishekwaghchaure/Desktop/Datasets/email/preprocessed_emails.csv')
df.head()

,file,headers,body,subject,from,to,clean_body,processed_body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,NaN,phillip.allen@enron.com,tim.belden@enron.com,Here is our forecast,"[['Here', 'forecast']]"
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,Re:,phillip.allen@enron.com,john.lavorato@enron.com,Traveling to have a business meeting takes the...,"[['Traveling', 'business', 'meeting', 'takes',..."
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Re: test,phillip.allen@enron.com,leah.arsdall@enron.com,test successful way to go,"[['test', 'successful', 'way', 'go']]"
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",NaN,phillip.allen@enron.com,randall.gay@enron.com,Randy Can you send me a schedule of the salary...,"[['Randy', 'Can', 'send', 'schedule', 'salary'..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,Re: Hello,phillip.allen@enron.com,greg.piper@enron.com,Lets shoot for Tuesday at 1145,"[['Lets', 'shoot', 'Tuesday', '1145']]"


In [3]:
df.isnull().sum()

file                  0
headers               0
body                  0
subject           19187
from                  0
to                 9143
clean_body          145
processed_body        0
dtype: int64

In [6]:
df = df.dropna(subset=['subject', 'processed_body'])
df.isnull().sum()

file                 0
headers              0
body                 0
subject              0
from                 0
to                8968
clean_body         125
processed_body       0
dtype: int64

In [8]:
df['headers']

1         Message-ID: <15464986.1075855378456.JavaMail.e...
2         Message-ID: <24216240.1075855687451.JavaMail.e...
4         Message-ID: <30922949.1075863688243.JavaMail.e...
5         Message-ID: <30965995.1075863688265.JavaMail.e...
7         Message-ID: <17189699.1075863688308.JavaMail.e...
                                ...                        
517396    Message-ID: <26807948.1075842029936.JavaMail.e...
517397    Message-ID: <25835861.1075842029959.JavaMail.e...
517398    Message-ID: <28979867.1075842029988.JavaMail.e...
517399    Message-ID: <22052556.1075842030013.JavaMail.e...
517400    Message-ID: <28618979.1075842030037.JavaMail.e...
Name: headers, Length: 498214, dtype: object

In [ ]:
class EmailDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len = 512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
